In [2]:
import os
import sys
import pathlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

path = pathlib.Path(os.getcwd())
module_path = str(path.parent) + '/'
sys.path.append(module_path)

from pymdp.agent import Agent
from pymdp import utils
from pymdp.maths import softmax
import copy

In [5]:
A_distance = np.ones((3, 5, 5)) / 3.0

In [6]:
A_distance[:,0,0] = np.eye(3)[0] # current is location 0, last was location 0, probability of '0 distance traveled' is 100%
A_distance[:,1,1] = np.eye(3)[0] # current is location 0, last was location 0, probability of '0 distance traveled' is 100%
A_distance[:,2,2] = np.eye(3)[0] # current is location 0, last was location 0, probability of '0 distance traveled' is 100%
A_distance[:,3,3] = np.eye(3)[0] # current is location 0, last was location 0, probability of '0 distance traveled' is 100%
A_distance[:,4,4] = np.eye(3)[0] # current is location 0, last was location 0, probability of '0 distance traveled' is 100%

In [7]:
# Starting location 0

# (1, 0) THIS EXISTS
# (1, 2) X
# (1, 3) X
# (1, 4) X

A_distance[:,1,0] = np.eye(3)[1] # current is location 1, last location was location 0, probability of '5 units distance traveled' is 100%

A_distance[:,4,0] = np.eye(3)[2] # current location is location 4, last location was location 0, probability of '20 units distance traveled' is 100%

# starting location 1
A_distance[:,2,1] = np.eye(3)[1] # current is location 0, last was location 0, probability of '5 distance traveled' is 100%

# Starting location 2
A_distance[:,3,2] = np.eye(3)[1] # current is location 0, last was location 0, probability of '5 distance traveled' is 100%

# Starting location 4
A_distance[:,3,4] = np.eye(3)[1] # current is location 0, last was location 0, probability of '5 distance traveled' is 100%


In [11]:
observation_names = ['0 distance', '5 distance', '20 distance']

In [25]:
curr_loc = 3
prev_loc = 4

edge_weight = observation_names[np.where(A_distance[:,curr_loc, prev_loc])[0][0]]
print(f'Observation given move from {prev_loc} to {curr_loc}: {edge_weight}')

Observation given move from 4 to 3: 5 distance


In [32]:
B = utils.obj_array(2)

B[0] = np.zeros((5, 5, 5)) # current location

B[1] = np.ones((5, 5, 5))  # previous location

In [36]:
qs_past = utils.obj_array(2)

In [37]:
qs_past[0] = np.eye(5)[2]
qs_past[1] = np.eye(5)[1]

In [39]:
qs_new = utils.obj_array(2)

In [ ]:
qs_new[0] = np.log(A_proprioception_current[obs_proprio_curr,:]) + np.log(B[0][:,:,last_action].dot(qs_past[0]))
qs_new[1] = np.log(A_proprioception_previous[obs_proprio_prev,:]) + np.log(B[1][:,:,0].dot(qs_past[1]))

### During policy evaluation / multi-step EFE integration, we need to tie together counterfactual beliefs about current location (imaginations about what my CURRENT location will be at t + tau) to counterfactual beliefs about previous location (imaginations about what my PREVIOUS location will be at t + tau). In order to do that, the idea is that we keep the hidden state factors statistically independent (B[0] does not depend on B[1]), but the space of allowable policies is reduced, such that if you're e.g. moving to Location 1 along hidden state factor 0, you can only move to consistent locations along hidden state factor 1. So like if the action along Hidden state factor 0 (current location) is "Move to Location 1", then policies that include that action can only have actions along Hidden state factor 1 (previous location) that are consistent with Location 1 being Current Location. 